In [7]:
import pandas as pd
import numpy as np
import random
import sqlite3 as sql
from datetime import datetime, timedelta
import os

In [8]:
conn = sql.connect("el_2.db")
cur = conn.cursor()

In [9]:
db_col = ["Index","EName","KPI1","KPI2","KPI3","KPI4","TimeStamp"]
kpi = ["KPI1","KPI2","KPI3","KPI4"]

In [11]:
def truncate_table(cmd,name):
    q = f"DROP TABLE {name}" if cmd == 'Drop' else f"DELETE FROM {name}"
    cur.execute(q)
    conn.commit()

def create_table():
    cur.execute('''
        CREATE TABLE IF NOT EXISTS Transactions(
            Id INTEGER PRIMARY KEY AUTOINCREMENT,
            EName TEXT NOT NULL,    
            KPI1 REAL DEFAULT 0,
            KPI2 REAL DEFAULT 0,
            KPI3 REAL DEFAULT 0,
            KPI4 REAL DEFAULT 0,
            TimeStamp TEXT NOT NULL
        );
    ''')
    conn.commit()

In [12]:
# truncate_table('Drop','Transactions')

In [13]:
def generate_daily_data(days):
    e_list = []
    k1 = []
    k2 = []
    k3 = []
    k4 = []
    t = []
    daily_seconds = 24*60*60
    timestamps = pd.date_range(start='2024-09-01 00:00:00', periods=days *daily_seconds, freq='s')
    for j in range(0,daily_seconds * days):
        for i in range(1,11):
            e = 'EL0' if i < 10 else 'EL' + str(i)
            k1Val = random.uniform(48,55)
            k2Val = random.uniform(28,35)
            k3Val = random.uniform(1,2.5)
            k4Val = random.uniform(490,510)
            tVal = timestamps[j]
            e_list.append(e)  
            t.append(tVal)
            k1.append(k1Val)
            k2.append(k2Val)
            k3.append(k3Val)
            k4.append(k4Val)
    data = {
        "EName":e_list,
        "KPI1":k1,
        "KPI2":k2,
        "KPI3":k3,
        "KPI4":k4,
        "TimeStamp":t
    }
    df = pd.DataFrame(data)
    return df

In [14]:
# if os.path.exists("./data.csv"):
#     print("file exists")
#     data = pd.read_csv("./data.csv")
# else:
#     print("generating data...")
data = generate_daily_data(15)
data.to_csv("./data_2.csv")
data.to_sql('Transactions',conn)

print(len(data))
data.head()

12960000


,EName,KPI1,KPI2,KPI3,KPI4,TimeStamp
0,EL0,48.645606,32.677675,1.979270,498.548756,2024-09-01
1,EL0,52.320402,33.039035,1.157428,491.097470,2024-09-01
2,EL0,50.041693,30.468551,1.571091,499.174003,2024-09-01
3,EL0,52.685471,29.935741,1.064911,508.555751,2024-09-01
4,EL0,53.641256,28.801699,1.642296,503.277794,2024-09-01


In [8]:
def get_date(date):
    currDate = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return currDate

In [9]:
def get_KPI_avg(data):
    res = []
    for i in range(1,11):
        # el = 'EL0' if i < 10 else 'EL'
        el = "EL" + str(i)
        el_data = data[data["EName"] == el][kpi]
        kpi_means = {j: np.round(el_data[j].mean(), 4).item() if not np.isnan(el_data[j].mean()) else 0.00 for j in kpi}
        res.append({el: kpi_means})
    kpi_avg = np.round((data[kpi].mean()).infer_objects(copy=False).fillna(0),4).to_dict()
    res.append({"total": kpi_avg})
    return res

In [10]:
def fetch_data_within_range(start,end):
    try:
        d = cur.execute(f"SELECT * from Transactions WHERE TimeStamp BETWEEN '{start}' AND '{end}';")
        selected_data = d.fetchall()
        return selected_data
    except sql.Error as e:
        print(e)
        return []

In [11]:
def create_df(data,col):
    data_df = pd.DataFrame(data,columns=col)
    return data_df

In [12]:
def get_prev_week_range(curr_week_start):
    prev_week_start = curr_week_start - timedelta(weeks=1)
    prev_week_end = curr_week_start - timedelta(seconds=1)
    return prev_week_start,prev_week_end

In [13]:
def get_current_week_range(current_date):
    current_week_start = current_date - timedelta(days=current_date.weekday())
    current_week_end = current_date + timedelta(days=6-current_date.weekday()) + timedelta(seconds=86399)
    return current_week_start,current_week_end

In [25]:
def get_current_hour_range(current_hour):
    prev_hour_end = current_hour - timedelta(hours=1)
    return current_hour,prev_hour_end

In [31]:
d = get_date('2024-09-05 00:10:00')

current_hour, prev_hour_end = d, d-timedelta(d.weekday())
current_week_start=d - timedelta(days=d.weekday())

prev_week_start,prev_week_end = get_prev_week_range(current_week_start)
prev_2_week_start,prev_2_week_end = get_prev_week_range(prev_week_start)

print("current: ",current_hour,prev_hour_end,current_week_start)
print("prev: ",prev_week_start,prev_week_end)
print("prev 2: ",prev_2_week_start,prev_2_week_end)

current:  2024-09-05 00:10:00 2024-09-02 00:10:00 2024-09-02 00:10:00
prev:  2024-08-26 00:10:00 2024-09-02 00:09:59
prev 2:  2024-08-19 00:10:00 2024-08-26 00:09:59


In [20]:
curr_week_data = fetch_data_within_range(current_week_start,d)
week1_data = fetch_data_within_range(prev_week_start,prev_week_end)
week2_data = fetch_data_within_range(prev_2_week_start,prev_2_week_end)

print(len(week1_data),len(week2_data))

864000 0


In [21]:
curr_df = create_df(curr_week_data,db_col)
week1_df = create_df(week1_data,db_col)
week2_df = create_df(week2_data,db_col)

print(week1_df.head(2))
print(week2_df.head(2))

   Index EName       KPI1       KPI2      KPI3        KPI4  \
0      0   EL1  50.472902  33.787028  1.698933  502.731440   
1      1   EL2  48.582651  32.049887  1.953103  490.827532   

             TimeStamp  
0  2024-09-01 00:00:00  
1  2024-09-01 00:00:00  
Empty DataFrame
Columns: [Index, EName, KPI1, KPI2, KPI3, KPI4, TimeStamp]
Index: []


In [22]:
curr_week = get_KPI_avg(curr_df)
week1_avg = get_KPI_avg(week1_df)
week2_avg = get_KPI_avg(week2_df)

In [23]:
curr_week

[{'EL1': {'KPI1': 50.5018, 'KPI2': 32.498, 'KPI3': 1.5001, 'KPI4': 497.5128}},
 {'EL2': {'KPI1': 50.4971, 'KPI2': 32.4992, 'KPI3': 1.5004, 'KPI4': 497.4925}},
 {'EL3': {'KPI1': 50.4987, 'KPI2': 32.4956, 'KPI3': 1.5, 'KPI4': 497.5014}},
 {'EL4': {'KPI1': 50.5034, 'KPI2': 32.5035, 'KPI3': 1.5002, 'KPI4': 497.5057}},
 {'EL5': {'KPI1': 50.5008, 'KPI2': 32.5033, 'KPI3': 1.4999, 'KPI4': 497.5126}},
 {'EL6': {'KPI1': 50.5013, 'KPI2': 32.5035, 'KPI3': 1.5, 'KPI4': 497.5006}},
 {'EL7': {'KPI1': 50.5016, 'KPI2': 32.495, 'KPI3': 1.5004, 'KPI4': 497.4841}},
 {'EL8': {'KPI1': 50.4977, 'KPI2': 32.5036, 'KPI3': 1.5002, 'KPI4': 497.5034}},
 {'EL9': {'KPI1': 50.499, 'KPI2': 32.501, 'KPI3': 1.4996, 'KPI4': 497.5022}},
 {'EL10': {'KPI1': 50.4981,
   'KPI2': 32.5015,
   'KPI3': 1.4996,
   'KPI4': 497.4941}},
 {'total': {'KPI1': 50.5, 'KPI2': 32.5004, 'KPI3': 1.5, 'KPI4': 497.501}}]